<a href="https://colab.research.google.com/github/GARYTJ29/matrixStructAnalysis/blob/main/Axial_include.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
try:
  from pandas import DataFrame
  f=0
except:
  f=1
def stdmat(L,E=1,A=1,I=1):
  ae= A*E/L
  a=4*E*I/L
  b=2*E*I/L
  c=6*E*I/L/L
  d=12*E*I/L/L/L

  mat = [[a,b,c,-c,0,0],
         [b,a,c,-c,0,0],
         [c,c,d,-d,0,0],
         [-c,-c,-d,d,0,0],
         [0,0,0,0,ae,-ae],
         [0,0,0,0,-ae,ae]]
  return mat
def tramat(c1,c2):
  dy = c2[1]-c1[1]
  dx = c2[0]-c1[0]
  if dx != 0:
    if dx>0:
      t = np.arctan(dy/dx)
    else:
      t = np.pi + np.arctan(dy/dx)
  else:
    t = np.pi/2
  c = round(np.cos(t),5)
  s = round(np.sin(t),5)
  mat = [[1,0,0,0,0,0],
        [0,1,0,0,0,0],
        [0,0,c,0,-s,0],
        [0,0,0,c,0,-s],
        [0,0,s,0,c,0],
        [0,0,0,s,0,c]]
  return mat

In [ ]:
joints = [ [],
    [0,0],[12,0],[0,6],[12,6],[0,12],[12,12]
]
E = 30
Ac = 93000
Ic = 720
Ag = 140000
Ig = 2430
Members = [ [],
    #columns
    [1,3,{'E':E,'A':Ac,'I':Ic}],
    [2,4,{'E':E,'A':Ac,'I':Ic}],
    [3,5,{'E':E,'A':Ac,'I':Ic}],
    [4,6,{'E':E,'A':Ac,'I':Ic}],
    #girders
    [3,4,{'E':E,'A':Ag,'I':Ig}],
    [5,6,{'E':E,'A':Ag,'I':Ig}],
    
]
JLoad = [0 , 
         #m
         0, 0, 0, 0, 0, 0,
         #y
         0, 0, 0, 0, 0, 0,
         #x
         0, 0, 60, 0, 30, 0
         ]

In [ ]:
tr = [[]]
locmat = [[]]
labelm = [{}]
leng = [0]
nj = len(joints)-1
for i in range(1,len(Members)):
  c1 = joints[Members[i][0]]
  c2 = joints[Members[i][1]]
  dy = c2[1]-c1[1]
  dx = c2[0]-c1[0]
  #transformation matrix
  tr += [tramat(c1,c2)]
  l = (dy*dy+dx*dx)**0.5
  leng.append(l)
  #local stiffness matrix
  locmat += [stdmat(l,Members[i][2]['E'],Members[i][2]['A'],Members[i][2]['I'])]
  #labelling Matrix
  m1 = Members[i][0]
  m2 = Members[i][1]
  labelm += [{m1:0,m2:1,(m1+nj):2,(m2+nj):3,(m1+2*nj):4,(m2+2*nj):5}]

In [ ]:
glomat = [[]]
for i in range(1,len(Members)):
  ttr = np.transpose(tr[i])
  mt = np.matmul(ttr,locmat[i])
  mt2 = np.matmul(mt,tr[i])
  glomat += [mt2]

In [ ]:
GLmat = np.zeros((3*nj,3*nj))
for i in range(1,len(Members)):
  for j in labelm[i]:
    for k in labelm[i]:
      GLmat[j-1][k-1] += glomat[i][labelm[i][j]][labelm[i][k]]

In [ ]:
Restr = [1,2,7,8,13,14]

In [ ]:
Restr.sort()
rst = list(map(lambda a: a-1 ,Restr))
Nrestr = []
for i in range(1,3*nj+1):
  if i not in Restr:
    Nrestr += [i]
nrst = list(map(lambda a: a-1 ,Nrestr))

In [ ]:
def ptl(P,L,a=-1):
  if a==-1:
    a=L/2
  b=L-a
  return [-P*a*b*b/L/L,P*a*a*b/L/L,-P*(3*a+b)*b*b/L/L/L,-P*(3*b+a)*a*a/L/L/L,0,0]
#ptl(40,5)
def udl(W,L):
  return [-W*L*L/12,W*L*L/12,-W*L/2,-W*L/2,0,0]
def tri(Wm,L,E='L'):
  
    a = [-Wm*L*L/30,Wm*L*L/20,-3*Wm*L/20,-7*Wm*L/20,0,0]
    if E=='R':
      return [a[1],a[0],a[3],a[2]]
    return a

In [ ]:
EleJ=[[],[0]*6,[0]*6,[0]*6,[0]*6,udl(12,leng[5]),udl(12,leng[6])]

In [ ]:
EleJG = [[]]
for i in range(1,len(Members)):
  ttr = np.transpose(tr[i])
  mt = np.dot(ttr,EleJ[i])
  EleJG += [mt]

In [ ]:
# JLoad = [0 ,
#          #m
#          0, 0, 0, 0, 0, 0,
#          #y
#          0, 0, 0, 0, 0, 0,
#          #x
#          0, 0, 60, 0, 30, 0
#          ]
for i in range(1,len(Members)):
  for j in labelm[i]:
    JLoad[j]+=EleJG[i][labelm[i][j]]

In [ ]:
Ju = []
for i in Nrestr:
  Ju.append(JLoad[i])
Jr = []
for i in Restr:
  Jr.append(JLoad[i])

In [ ]:
Kuu = np.array(GLmat)
Kuu = np.delete(Kuu, rst,axis=1)
Kuu = np.delete(Kuu, rst,axis=0)
kuuinv = np.linalg.inv(Kuu)
deluu = np.dot(kuuinv,Ju)

In [ ]:
delmat = [0]*(3*nj+1)
for i in range(len(Nrestr)):
  delmat[Nrestr[i]]=deluu[i]

In [ ]:
print("\nLocal Stiffness Matrix (Global Coordintes):")
for i in range(1,len(Members)):
  print("For Member", i , ":")
  print(np.array(glomat[i]))


Local Stiffness Matrix (Global Coordintes):
For Member 1 :
[[  14400.    7200.       0.       0.   -3600.    3600.]
 [   7200.   14400.       0.       0.   -3600.    3600.]
 [      0.       0.  465000. -465000.       0.       0.]
 [      0.       0. -465000.  465000.       0.       0.]
 [  -3600.   -3600.       0.       0.    1200.   -1200.]
 [   3600.    3600.       0.       0.   -1200.    1200.]]
For Member 2 :
[[  14400.    7200.       0.       0.   -3600.    3600.]
 [   7200.   14400.       0.       0.   -3600.    3600.]
 [      0.       0.  465000. -465000.       0.       0.]
 [      0.       0. -465000.  465000.       0.       0.]
 [  -3600.   -3600.       0.       0.    1200.   -1200.]
 [   3600.    3600.       0.       0.   -1200.    1200.]]
For Member 3 :
[[  14400.    7200.       0.       0.   -3600.    3600.]
 [   7200.   14400.       0.       0.   -3600.    3600.]
 [      0.       0.  465000. -465000.       0.       0.]
 [      0.       0. -465000.  465000.       0.       

In [ ]:
print("\nComplete structure stiffness matrix:")
#import sys
if f:
  print(np.array(GLmat))
else:
  print(DataFrame(GLmat).to_string(index=False,header=False))


Complete structure stiffness matrix:
14400.0     0.0  7200.0     0.0     0.0     0.0       0.0       0.0       0.00       0.00       0.00       0.00 -3600.0     0.0    3600.0       0.0       0.0       0.0
    0.0 14400.0     0.0  7200.0     0.0     0.0       0.0       0.0       0.00       0.00       0.00       0.00     0.0 -3600.0       0.0    3600.0       0.0       0.0
 7200.0     0.0 53100.0 12150.0  7200.0     0.0       0.0       0.0    3037.50   -3037.50       0.00       0.00 -3600.0     0.0       0.0       0.0    3600.0       0.0
    0.0  7200.0 12150.0 53100.0     0.0  7200.0       0.0       0.0    3037.50   -3037.50       0.00       0.00     0.0 -3600.0       0.0       0.0       0.0    3600.0
    0.0     0.0  7200.0     0.0 38700.0 12150.0       0.0       0.0       0.00       0.00    3037.50   -3037.50     0.0     0.0   -3600.0       0.0    3600.0       0.0
    0.0     0.0     0.0  7200.0 12150.0 38700.0       0.0       0.0       0.00       0.00    3037.50   -3037.50     0.0   

In [ ]:
print("\nUnrestrained stiffness sub-matrix:")
if f:
  print(np.array(Kuu))
else:
  print(DataFrame(Kuu).to_string(index=False,header=False))


Unrestrained stiffness sub-matrix:
53100.0 12150.0  7200.0     0.0    3037.50   -3037.50       0.00       0.00       0.0       0.0    3600.0       0.0
12150.0 53100.0     0.0  7200.0    3037.50   -3037.50       0.00       0.00       0.0       0.0       0.0    3600.0
 7200.0     0.0 38700.0 12150.0       0.00       0.00    3037.50   -3037.50   -3600.0       0.0    3600.0       0.0
    0.0  7200.0 12150.0 38700.0       0.00       0.00    3037.50   -3037.50       0.0   -3600.0       0.0    3600.0
 3037.5  3037.5     0.0     0.0  930506.25    -506.25 -465000.00       0.00       0.0       0.0       0.0       0.0
-3037.5 -3037.5     0.0     0.0    -506.25  930506.25       0.00 -465000.00       0.0       0.0       0.0       0.0
    0.0     0.0  3037.5  3037.5 -465000.00       0.00  465506.25    -506.25       0.0       0.0       0.0       0.0
    0.0     0.0 -3037.5 -3037.5       0.00 -465000.00    -506.25  465506.25       0.0       0.0       0.0       0.0
    0.0     0.0 -3600.0     0.0     

In [ ]:
print("\nInverse of unrestrained stiffness sub-matrix:")
if f:
  print(np.array(kuuinv))
else:
  print(DataFrame(kuuinv).to_string(index=False,header=False))


Inverse of unrestrained stiffness sub-matrix:
 2.439061e-05 -1.259445e-06 -2.407307e-06  4.540463e-06 -1.642881e-07  1.642881e-07 -1.778352e-07  1.778352e-07 -3.471464e-05 -3.467886e-05 -0.000073 -0.000073
-1.259445e-06  2.439061e-05  4.540463e-06 -2.407307e-06 -1.642881e-07  1.642881e-07 -1.778352e-07  1.778352e-07 -3.467886e-05 -3.471464e-05 -0.000073 -0.000073
-2.407307e-06  4.540463e-06  3.243171e-05 -7.167382e-06 -1.778352e-07  1.778352e-07 -3.421234e-07  3.421234e-07 -3.098399e-06 -3.301070e-06 -0.000044 -0.000044
 4.540463e-06 -2.407307e-06 -7.167382e-06  3.243171e-05 -1.778352e-07  1.778352e-07 -3.421234e-07  3.421234e-07 -3.301070e-06 -3.098399e-06 -0.000044 -0.000044
-1.642881e-07 -1.642881e-07 -1.778352e-07 -1.778352e-07  2.150326e-06  2.119847e-10  2.150308e-06  2.294648e-10  4.928644e-07  4.928644e-07  0.000002  0.000002
 1.642881e-07  1.642881e-07  1.778352e-07  1.778352e-07  2.119847e-10  2.150326e-06  2.294648e-10  2.150308e-06 -4.928644e-07 -4.928644e-07 -0.000002 -0.

In [ ]:
print("\nJoint Load Vector is:")
#for x in JLoad:
#  print(x)
print(np.array(JLoad[1::]).reshape(len(JLoad)-1,1))


Joint Load Vector is:
[[   0.]
 [   0.]
 [-144.]
 [ 144.]
 [-144.]
 [ 144.]
 [   0.]
 [   0.]
 [ -72.]
 [ -72.]
 [ -72.]
 [ -72.]
 [   0.]
 [   0.]
 [  60.]
 [   0.]
 [  30.]
 [   0.]]


In [ ]:
print("\nUnrestrained displacements are:")
for i in range(3,nj+1):
  print(f"Joint {i} : θ = {delmat[i]} rad  | δy = {delmat[i+nj]} m  | δx =  {delmat[i+2*nj]} m")


Unrestrained displacements are:
Joint 3 : θ = -0.006955245834687568 rad  | δy = -0.0002345285189414512 m  | δx =  0.05029753898926292 m
Joint 4 : θ = -0.0015639585245556973 rad  | δy = -0.0003848263197682264 m  | δx =  0.050260074088463745 m
Joint 5 : θ = -0.006219081600949511 rad  | δy = -0.0003697043389141557 m  | δx =  0.08017958551446855 m
Joint 6 : θ = 0.003177356284015537 rad  | δy = -0.0005593279191503605 m  | δx =  0.0800608165917884 m


In [ ]:
jvector = [0]*(3*nj+1)
print("\nForces on Each Member are (Fy,Fx are shown in global Coordinate system):")
for i in range(1,len(Members)):
  print("Member ",i,":")
  Tempdel = []
  c1 = Members[i][0]
  c2 = Members[i][1]
  for j in [c1,c2,c1+nj,c2+nj,c1+2*nj,c2+2*nj]:
    Tempdel.append(delmat[j])
  Ftemp = np.dot(glomat[i],Tempdel) - EleJG[i]
  #Ftemp = list(map(lambda a: round(a,3),Ftemp))
  print("\tDisplacement Vector is ",Tempdel)
  print("\tMoment Vector is ", Ftemp)
  print(f"\tJoint {c1} : M = {round(Ftemp[0],3)} kNm | Fy = {round(Ftemp[2],3)} kN | Fx = {round(Ftemp[4],3)} kN")
  print(f"\tJoint {c2} : M = {round(Ftemp[1],3)} kNm | Fy = {round(Ftemp[3],3)} kN | Fx = {round(Ftemp[5],3)} kN")
  jvector[c1] +=Ftemp[0]
  jvector[nj + c1] +=Ftemp[2]
  jvector[2*nj +c1] +=Ftemp[4]
  jvector[c2] +=Ftemp[1]
  jvector[1*nj +c2] +=Ftemp[3]
  jvector[2*nj +c2] +=Ftemp[5]



Forces on Each Member are (Fy,Fx are shown in global Coordinate system):
Member  1 :
	Displacement Vector is  [0, -0.006955245834687568, 0, -0.0002345285189414512, 0, 0.05029753898926292]
	Moment Vector is  [ 130.99337035   80.91560034  109.05576131 -109.05576131  -35.31816178
   35.31816178]
	Joint 1 : M = 130.993 kNm | Fy = 109.056 kN | Fx = -35.318 kN
	Joint 3 : M = 80.916 kNm | Fy = -109.056 kN | Fx = 35.318 kN
Member  2 :
	Displacement Vector is  [0, -0.0015639585245556973, 0, -0.0003848263197682264, 0, 0.050260074088463745]
	Moment Vector is  [ 169.67576534  158.41526396  178.94423869 -178.94423869  -54.68183822
   54.68183822]
	Joint 2 : M = 169.676 kNm | Fy = 178.944 kN | Fx = -54.682 kN
	Joint 4 : M = 158.415 kNm | Fy = -178.944 kN | Fx = 54.682 kN
Member  3 :
	Displacement Vector is  [-0.006955245834687568, -0.006219081600949511, -0.0002345285189414512, -0.0003697043389141557, 0.05029753898926292, 0.08017958551446855]
	Moment Vector is  [-37.35756006 -32.05717757  62.8567562

In [ ]:
Kru = np.array(GLmat)
Kru = np.delete(Kru,nrst, axis=0)
Kru = np.delete(Kru,rst, axis=1)

#print(Kru)
#kruinv = np.linalg.inv(Kru)
Fr = np.dot(Kru,deluu)-Jr
print("\nReaction Forces:")
Frar = [{} for _ in range((nj+1))]
for i in range(len(Restr)):
  if(Restr[i]<=nj):
   Frar[Restr[i]]['M'] = Fr[i]
  elif Restr[i]<=2*nj:
    Frar[Restr[i]-nj]['Fy'] = Fr[i]
  else:
    Frar[Restr[i]-2*nj]['Fx'] = Fr[i]
#print(Frt)

for i in range(1,nj+1):
  if Frar[i]:
    
    print(f"Joint {i} :",end="\n")
    if 'Fy' in Frar[i]:
      print(f"\t M = {round(Frar[i]['M'],3)} kNm",end=" ")
    if 'Fy' in Frar[i] and 'M' in Frar[i]:
      print("|",end=" ")
    else:
      print("\t",end=" ")
    if 'Fy' in Frar[i]:
      print(f"Fy = {round(Frar[i]['Fy'],3)} kN",end=" ")
    if 'Fy' in Frar[i] and 'Fx' in Frar[i]:
      print("|",end=" ")
    else:
      print("\t",end=" ")
    if 'Fx' in Frar[i]:
      print(f" Fx = {round(Frar[i]['Fx'],3)} kN",end=" ")
    print("")


Reaction Forces:
Joint 1 :
	 M = 130.993 kNm | Fy = 109.056 kN |  Fx = -35.318 kN 
Joint 2 :
	 M = 169.676 kNm | Fy = 178.944 kN |  Fx = -54.682 kN 


In [ ]:
print("\nJoint Force Vector is :\n")
print(np.array(list(map(np.around,jvector[1::],[3]*(3*nj)))).reshape(3*nj,1))


Joint Load Vecor is :

[[130.993]
 [169.676]
 [  0.   ]
 [  0.   ]
 [ -0.   ]
 [ -0.   ]
 [109.056]
 [178.944]
 [ -0.   ]
 [ -0.   ]
 [  0.   ]
 [  0.   ]
 [-35.318]
 [-54.682]
 [ 60.   ]
 [  0.   ]
 [ 30.   ]
 [ -0.   ]]
